In [1]:

import os
import sys
sys.path.append('/Users/neil_panchal/Desktop/tensorflow/models/research/')
sys.path.append('/Users/neil_panchal/Desktop/tensorflow/models/research/object_detection/')
from glob import glob
from PIL import Image
from PIL import Image
from PIL import ImageDraw
from PIL import ImageColor
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from collections import defaultdict
from utils import label_map_util
from utils import visualization_utils as vis_util
import random

%matplotlib inline

/Users/neil_panchal/anaconda3/envs/Neil/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


## Load Graph
Function to load from to memory

In [2]:

def load_graph(graph_file):
    """Loads a frozen inference graph"""
    graph = tf.Graph()
    with graph.as_default():
        od_graph_def = tf.GraphDef()
        with tf.gfile.GFile(graph_file, 'rb') as fid:
            serialized_graph = fid.read()
            od_graph_def.ParseFromString(serialized_graph)
            tf.import_graph_def(od_graph_def, name='')
    return graph


def load_image_into_numpy_array(image):
    (im_width, im_height) = image.size
    return np.array(image.getdata()).reshape((im_height, im_width, 3)).astype(np.uint8)

# Colors (one for each class)
cmap = ImageColor.colormap
print("Number of colors =", len(cmap))
COLOR_LIST = sorted([c for c in cmap.keys()])

#
# Utility funcs
#

def filter_boxes(min_score, boxes, scores, classes):
    """Return boxes with a confidence >= `min_score`"""
    n = len(classes)
    idxs = []
    for i in range(n):
        if scores[i] >= min_score:
            idxs.append(i)
    
    filtered_boxes = boxes[idxs, ...]
    filtered_scores = scores[idxs, ...]
    filtered_classes = classes[idxs, ...]
    return filtered_boxes, filtered_scores, filtered_classes

def to_image_coords(boxes, height, width):
    """
    The original box coordinate output is normalized, i.e [0, 1].
    
    This converts it back to the original coordinate based on the image
    size.
    """
    box_coords = np.zeros_like(boxes)
    box_coords[:, 0] = boxes[:, 0] * height
    box_coords[:, 1] = boxes[:, 1] * width
    box_coords[:, 2] = boxes[:, 2] * height
    box_coords[:, 3] = boxes[:, 3] * width
    
    return box_coords

def draw_boxes(image, boxes, classes, thickness=4):
    """Draw bounding boxes on the image"""
    draw = ImageDraw.Draw(image)
    for i in range(len(boxes)):
        bot, left, top, right = boxes[i, ...]
        class_id = int(classes[i])
        color = COLOR_LIST[class_id]
        draw.line([(left, top), (left, bot), (right, bot), (right, top), (left, top)], width=thickness, fill=color)
        
def load_graph(graph_file):
    """Loads a frozen inference graph"""
    graph = tf.Graph()
    with graph.as_default():
        od_graph_def = tf.GraphDef()
        with tf.gfile.GFile(graph_file, 'rb') as fid:
            serialized_graph = fid.read()
            od_graph_def.ParseFromString(serialized_graph)
            tf.import_graph_def(od_graph_def, name='')
    return graph

Number of colors = 148


### Paths to Dataset and Saved Models

In [3]:


PATH_TO_GRAPH = 'output_models/frozen_inference_graph.pb' ## load SSD trained on udacity's parking lot images
PATH_TO_LABELS = '/Users/neil_panchal/Desktop/Udacity/alex-lechner-udacity-traffic-light-dataset/udacity_label_map.pbtxt'
NUM_CLASSES = 4

In [4]:
detection_graph = load_graph(PATH_TO_GRAPH)


In [5]:
label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
category_index = label_map_util.create_category_index(categories)
print(category_index)

{1: {'id': 1, 'name': 'Green'}, 2: {'id': 2, 'name': 'Red'}, 3: {'id': 3, 'name': 'Yellow'}, 4: {'id': 4, 'name': 'off'}}


In [6]:

IMAGE_SIZE = (12, 8)

In [7]:
PATH_TO_IMGS = '/Users/neil_panchal/Desktop/Udacity/alex-lechner-udacity-traffic-light-dataset/udacity_testarea_rgb'
TEST_IMGS = glob(os.path.join(PATH_TO_IMGS, r'*.jpg'))
random.shuffle(TEST_IMGS)

# with detection_graph.as_default():
#     with tf.Session(graph=detection_graph) as sess:
#         image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
#         detect_boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
#         detect_scores = detection_graph.get_tensor_by_name('detection_scores:0')
#         detect_classes = detection_graph.get_tensor_by_name('detection_classes:0')
#         num_detections = detection_graph.get_tensor_by_name('num_detections:0')
        
#         for idx, img_path in enumerate(TEST_IMGS):
#             random.shuffle(TEST_IMGS)
#             image = Image.open(img_path)
#             image_np = load_image_into_numpy_array(image)
#             image_expanded = np.expand_dims(image_np, axis=0)
            
#             (boxes, scores, classes, num) = sess.run(
#                 [detect_boxes, detect_scores, detect_classes, num_detections],
#                 feed_dict={image_tensor: image_expanded})
#             draw_boxes(image, box_coords, classes)
#             print('SCORES')
#             print(scores[0])
#             print('CLASSES')
#             print(classes[0])
#             print('BOXES')
#             print(boxes)
            
#             vis_util.visualize_boxes_and_labels_on_image_array(
#                 image_np, 
#                 np.squeeze(boxes),
#                 np.squeeze(classes).astype(np.int32),
#                 np.squeeze(scores),
#                 category_index,
#                 use_normalized_coordinates=False,
#                 max_boxes_to_draw=3,
#                 line_thickness=4)
#             plt.figure(figsize=IMAGE_SIZE)
#             plt.imshow(image_np)
#             plt.show()
            
#             if idx == 10:
#                 break
image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
detect_boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
detect_scores = detection_graph.get_tensor_by_name('detection_scores:0')
detect_classes = detection_graph.get_tensor_by_name('detection_classes:0')
num_detections = detection_graph.get_tensor_by_name('num_detections:0')
with tf.Session(graph=detection_graph) as sess:                
    # Actual detection.
    image_expanded = np.expand_dims(image_np, axis=0)
    (boxes, scores, classes) = sess.run([detect_boxes, detect_scores, detect_classes], 
                                        feed_dict={image_tensor: image_np})

    # Remove unnecessary dimensions
    boxes = np.squeeze(boxes)
    scores = np.squeeze(scores)
    classes = np.squeeze(classes)

    confidence_cutoff = 0.8
    # Filter boxes with a confidence score less than `confidence_cutoff`
    boxes, scores, classes = filter_boxes(confidence_cutoff, boxes, scores, classes)

    # The current box coordinates are normalized to a range between 0 and 1.
    # This converts the coordinates actual location on the image.
    width, height = image.size
    box_coords = to_image_coords(boxes, height, width)

    # Each class with be represented by a differently colored box
    draw_boxes(image, box_coords, classes)

    plt.figure(figsize=(12, 8))
    plt.imshow(image) 

NameError: name 'image_np' is not defined

In [ ]:
file_path = '/Users/neil_panchal/Desktop/Udacity/alex-lechner-udacity-traffic-light-dataset/jpg_simulator_train.record'
tfrecord_dataset = tf.data.TFRecordDataset(file_path)

In [ ]:
PATH_TO_GRAPH = 'output_models/output_models/frozen_inference_graph.pb' ## load SSD trained on udacity's simulator images
detection_graph = load_graph(PATH_TO_GRAPH)

In [ ]:
PATH_TO_IMGS = '/Users/neil_panchal/Desktop/Udacity/alex-lechner-udacity-traffic-light-dataset/simulator_dataset_rgb'
subfolder = ['Green', 'Red', 'Yellow', 'Unknown']
TEST_IMGS = []
for i in subfolder:
    TEST_IMGS += glob(os.path.join(os.path.join(PATH_TO_IMGS, i), r'*.jpg'))
random.shuffle(TEST_IMGS)

with detection_graph.as_default():
    with tf.Session(graph=detection_graph) as sess:
        image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
        detect_boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
        detect_scores = detection_graph.get_tensor_by_name('detection_scores:0')
        detect_classes = detection_graph.get_tensor_by_name('detection_classes:0')
        num_detections = detection_graph.get_tensor_by_name('num_detections:0')
        
        for idx, img_path in enumerate(TEST_IMGS):
            image = Image.open(img_path)
            image_np = load_image_into_numpy_array(image)
            image_expanded = np.expand_dims(image_np, axis=0)
            
            (boxes, scores, classes, num) = sess.run(
                [detect_boxes, detect_scores, detect_classes, num_detections],
                feed_dict={image_tensor: image_expanded})
            
            print('SCORES')
            print(scores[0])
            print('CLASSES')
            print(classes[0])
            
            vis_util.visualize_boxes_and_labels_on_image_array(
                image_np, 
                np.squeeze(boxes),
                np.squeeze(classes).astype(np.int32),
                np.squeeze(scores),
                category_index,
                use_normalized_coordinates=True,
                max_boxes_to_draw=5,
                line_thickness=8)
            plt.figure(figsize=IMAGE_SIZE)
            plt.imshow(image_np)
            plt.show()
            
            if idx == 10:
                break